<a href="https://colab.research.google.com/github/haricharanvihari/extensive_viz/blob/master/S4_EVA4_Haricharan_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #Convolution2D - This actually performs the 2D convolution on the input and produce the convoluted output.
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.dropout1 = nn.Dropout2d(0.25) #Kind of regularisation. It applies Dropout to the input, randomly setting a fraction of input units to 0 at each update during training, which helps prevent overfitting.
        self.conv2_bn = nn.BatchNorm2d(20) #Batch Normalization is that small component that sits between the layers of the neural network, continuously taking the ouput of a particular layer and normalizing it before sending it across to the next layer as input.
        self.pool1 = nn.MaxPool2d(2, 2) # Max-pooling helps in extracting low-level features like edges, points.
        self.conv3 = nn.Conv2d(64, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(20)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(64, 32, 3)
        self.avgpool = nn.AvgPool2d(3, stride=1) #While Avg-pooling helps in extracting smooth features.
        self.conv6 = nn.Conv2d(32, 64, 3)
        self.avgpool = nn.AvgPool2d(3, stride=1)
        self.conv7 = nn.Conv2d(64, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # ReLU converts all the pixel values between [-1,1] to [0,1]
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x) #After applying softmax, each component will be in the interval (0,1), and the components will add up to 1, so that they can be interpreted as probabilities, & highest probability will be considered for output.

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]          18,464
            Conv2d-5           [-1, 64, 14, 14]          18,496
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]          18,464
            Conv2d-8             [-1, 64, 3, 3]          18,496
            Conv2d-9             [-1, 10, 1, 1]           5,770
Total params: 98,506
Trainable params: 98,506
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.85
Params size (MB): 0.38
Estimated Total Size (MB): 1.23
---------------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64 # The number of images passed through the network for each epoch.

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20): #One epoch is when an entire dataset is passed both forward and backward through the neural network only once
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.5220, Accuracy: 8788/10000 (87.88%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.5058, Accuracy: 8864/10000 (88.64%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0366, Accuracy: 9920/10000 (99.20%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0396, Accuracy: 9921/10000 (99.21%)



loss=2.384185791015625e-07 batch_id=937: 100%|██████████| 938/938 [00:13<00:00, 68.88it/s]



Test set: Average loss: 0.0379, Accuracy: 9933/10000 (99.33%)

